# Miguel Angel Paz Camacho

# Hackaton Mentes Bayesianas

# 1 Modelos de Lenguaje Neuronales

In [1]:
import os
import time
import shutil
import random
from typing import Tuple
from argparse import Namespace
import matplotlib.pyplot as plt

#Preprosacimiento

import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
import pandas as pd
import numpy as np

#PyTorch

from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F


#scikit-learn

from sklearn.metrics import accuracy_score

#Para lectura de embedings

from gensim.models import KeyedVectors


In [2]:
seed = 1111
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False # Instrucción para optimizar las convoluciones. Si esta en False permite
#un número de inputs cambiante en lugar de uno fijo.

In [61]:
from sklearn.model_selection import train_test_split

Data_pd = pd.read_csv('claudia.csv', header=None, sep="\n\n")
print(Data_pd.columns)
#print(X_train.columns)

Tweets = Data_pd[0].tolist()
#print(Tweets)
#Tweets = Data_pd["tweetText"].tolist()

X_train, X_val = train_test_split(Tweets, test_size=0.2, random_state=42)

print(X_train)
#X_val = pd.read_csv('mex20_val.txt', sep='\r\n', engine='python', header=None).loc[:,0].values.tolist()
#print(X_val)
#embeddings_model = KeyedVectors.load_word2vec_format('word2vec_col.txt', binary=False)
#print(embeddings)

Index([0], dtype='int64')
['premio a la innovación a la excelencia por la asociación de comercio y contratación mundial', 'muy buena pregunta', 'sí sí quisiéramos que hubiera justicia en los relojes por favor', 'sí cómo no', 'siempre he estado del lado de las víctimas y de la justicia durante toda mi vida el caso rébsamen como seguramente otros que va a plantear la candidata del prian los he aclarado inclusive ha habido justicia lo que es deleznable realmente es lucrar con el dolor de las personas nunca seremos así', 'sí son varios temas que planteamos y además no estoy de acuerdo con lo que planteas denise porque repito se han ahorrado 24 billones de pesos si no no se pudiera haber hecho lo que se ha hecho en estos casi seis años del presidente lópez obrador', 'el gobierno de méxico tiene diversos programas en centroamérica y hay que seguir insistiendo con estados unidos porque el objetivo de los migrantes es ir a estados unidos y lo vamos a seguir insistiendo que la mejor forma de at

C:\Users\Miguel\AppData\Local\Temp\ipykernel_4628\541386454.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  Data_pd = pd.read_csv('claudia.csv', header=None, sep="\n\n")


## Trabajo Practica

### Entrenamiento

In [62]:
args = Namespace()
args.N = 4

In [63]:
class NgramData():

    def __init__(self, N: int, vocab_max: int=5000, tokenizer=None, embeddings_model=None):
        self.tokenizer = tokenizer if tokenizer else self.default_tokenizer
        self.punct = set(['.', ',', ';', ':', '-', '^', '»', '!', '¡', '¿', '?', '"', '\'', '...', '<url>', '*', '@usuario'])
        self.N = N
        self.vocab_max = vocab_max
        self.UNK = "<unk>"
        self.SOS = '<s>'
        self.EOS = '</s>'
        self.embeddings_model = embeddings_model

    def get_vocab_size(self) -> int:
        return len(self.vocab)

    def default_tokenizer(self, doc:str) -> list:
        return doc.split(" ")

    def remove_word(self, word: str) -> bool:
        word = word.lower()
        is_punct = True if word in self.punct else False
        is_digit = word.isnumeric()
        return is_punct or is_digit

    def get_vocab(self, corpus: list) -> set:
        freq_dist = FreqDist([w.lower() for sentence in corpus\
                                        for w in self.tokenizer(sentence)\
                                        if not self.remove_word(w)])

        sorted_words = self.sortFreqDict(freq_dist)[:self.vocab_max-3] #Incluimos al vocabulario las palabras "<unk>" '<s>' '</s>'
        return set(sorted_words)

    def sortFreqDict(self, freq_dist) -> list:
        freq_dict = dict(freq_dist)
        return sorted(freq_dict, key=freq_dict.get, reverse=True)

    def fit(self, corpus: list) -> None:
        self.vocab = self.get_vocab(corpus)
        self.vocab.add(self.UNK)
        self.vocab.add(self.SOS)
        self.vocab.add(self.EOS)

        self.w2id = {}
        self.id2w = {}

        if self.embeddings_model is not None:
            self.embeddings_matrix = np.empty([len(self.vocab), self.embeddings_model.vector_size])

        id = 0
        for doc in corpus:
            for word in self.tokenizer(doc):
                word_ = word.lower()
                if word_ in self.vocab and not word_ in self.w2id:
                    self.w2id[word_] = id
                    self.id2w[id] = word_

                    if self.embeddings_model is not None:
                        if word_ in self.embeddings_model:
                            self.embeddings_matrix[id] = self.embeddings_model[word_]
                        else:
                            self.embeddings_matrix[id] = np.random.rand(self.embeddings_model.vector_size)

                    id +=1


        self.w2id.update(
            {
                self.UNK: id,
                self.SOS: id+1,
                self.EOS: id+2
            }
        )
        self.id2w.update(
            {
                id: self.UNK,
                id+1: self.SOS,
                id+2: self.EOS
            }
        )

    def transform(self, corpus: list) -> Tuple[np.ndarray, np.ndarray]:

        X_ngrams = []
        y = []

        for doc in corpus:
            doc_ngram = self.get_ngram_doc(doc)
            for words_window in doc_ngram:
                words_window_ids = [self.w2id[w] for w in words_window]
                X_ngrams.append(list(words_window_ids[:-1]))
                y.append(words_window_ids[-1])

        return np.array(X_ngrams), np.array(y)

    def get_ngram_doc(self, doc: str) -> list:
        doc_tokens = self.tokenizer(doc)
        doc_tokens = self.replace_unk(doc_tokens)
        doc_tokens = [w.lower() for w in doc_tokens]
        doc_tokens = [self.SOS]*(self.N-1) + doc_tokens + [self.EOS]
        return list(ngrams(doc_tokens, self.N))

    def replace_unk(self, doc_tokens: list) -> list:
        for i, token in enumerate(doc_tokens):
            if token.lower() not in self.vocab:
                doc_tokens[i] = self.UNK

        return doc_tokens

In [64]:
tk = TweetTokenizer()
ngram_data = NgramData(args.N, 5000, tk.tokenize)
ngram_data.fit(X_train)

In [65]:
print(f'Vocab size: {ngram_data.get_vocab_size()}')

Vocab size: 873


In [66]:
X_ngram_train, y_ngram_train = ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = ngram_data.transform(X_val)

In [67]:
print(f'Training observations: X: {X_ngram_train.shape}, y: {y_ngram_train.shape}')
print(f'Validation observations: X: {X_ngram_val.shape}, y: {y_ngram_val.shape}')

Training observations: X: (3398, 3), y: (3398,)
Validation observations: X: (739, 3), y: (739,)


In [68]:
[[ngram_data.id2w[w] for w in tw] for tw in X_ngram_train[:10]]

[['<s>', '<s>', '<s>'],
 ['<s>', '<s>', 'premio'],
 ['<s>', 'premio', 'a'],
 ['premio', 'a', 'la'],
 ['a', 'la', 'innovación'],
 ['la', 'innovación', 'a'],
 ['innovación', 'a', 'la'],
 ['a', 'la', 'excelencia'],
 ['la', 'excelencia', 'por'],
 ['excelencia', 'por', 'la']]

In [69]:
y_ngram_train

array([  0,   1,   2, ..., 872,  15, 872])

In [70]:
[ngram_data.id2w[w] for w in y_ngram_train[:10]]

['premio',
 'a',
 'la',
 'innovación',
 'a',
 'la',
 'excelencia',
 'por',
 'la',
 'asociación']

In [71]:
#Set batch size in args
args.batch_size = 64

#Num workers
args.num_workers = 2

#Train
train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype=torch.int64),
                              torch.tensor(y_ngram_train, dtype=torch.int64))

train_loader = DataLoader(train_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=True)

#Val
val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype=torch.int64),
                            torch.tensor(y_ngram_val, dtype=torch.int64))

val_loader = DataLoader(val_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=False)

In [72]:
batch = next(iter(train_loader))
print(f'X shape: {batch[0].shape}')
print(f'Y shape: {batch[1].shape}')

X shape: torch.Size([64, 3])
Y shape: torch.Size([64])


In [73]:
#Vocab size
args.vocab_size = ngram_data.get_vocab_size()

#Dimension of word embeddings
args.d = 50

#Dimension for hidden layer
args.d_h = 100

#Dropout
args.dropout = 0.1

In [74]:
class NeuralLM(nn.Module):

    def __init__(self, args):
        super(NeuralLM, self).__init__()

        self.window_size = args.N-1
        self.embedding_dim = args.d

        self.emb = nn.Embedding(args.vocab_size, args.d)
        self.fc1 = nn.Linear(args.d*(args.N-1), args.d_h)
        self.drop1 = nn.Dropout(p=args.dropout)
        self.fc2 = nn.Linear(args.d_h, args.vocab_size, bias=False)

    def forward(self, x):
        x = self.emb(x)
        x = x.view(-1, self.window_size*self.embedding_dim)
        h = F.relu(self.fc1(x) )
        h = self.drop1(h)
        return self.fc2(h)


In [75]:
def get_preds(raws_logits):
    probs = F.softmax(raws_logits.detach(), dim=1)
    y_pred = torch.argmax(probs, dim=1).cpu().numpy()

    return y_pred

In [76]:
def model_eval(data, model, gpu=False):
    with torch.no_grad():
        preds, tgts = [], []
        for window_words, labels in data:
            if gpu:
                window_words = window_words.cuda()

            outputs = model(window_words)

            #Obtener prediccion
            y_pred = get_preds(outputs)

            tgt = labels.numpy()
            tgts.append(tgt)
            preds.append(y_pred)

    tgts = [e for l in tgts for e in l]
    preds = [e for l in preds for e in l]

    return accuracy_score(tgts, preds)

In [77]:
def save_checkpoint(state, is_best, checkpoint_path, filename="checkpoint.pt"):
    filename = os.path.join(checkpoint_path, filename)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join(checkpoint_path, "model_best.pt"))

In [78]:
#Modelo Hyperparametros

args.vocab_size = ngram_data.get_vocab_size()
args.d = 100 #Dimension del embedding
args.d_h = 200 #Dimension de la capa oculta
args.dropout = 0.1

#Training Hyperparametros
args.lr = 2.3e-1
args.num_epochs = 100
args.patience = 20

#Scheduler Hyperparametros
args.lr_patience = 10
args.lr_factor = 0.5

#Saving directory
args.savedir = 'model'
os.makedirs(args.savedir, exist_ok=True)

#Crear modelo
model = NeuralLM(args)

#Enviar a GPU
args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
    model.cuda()

#Perdida, Optimizador y Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, "min",
                patience=args.lr_patience,
                verbose=True,
                factor=args.lr_factor
            )

c:\Users\Miguel\anaconda3\envs\machine-learning\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [79]:
start_time = time.time()
best_metric = 0
metric_history = []
train_metric_history = []

for epoch in range(args.num_epochs):
    epoch_start_time = time.time()
    loss_epoch = []
    training_metric = []
    model.train()

    for window_words, labels in train_loader:

        #Si hay GPU
        if args.use_gpu:
            window_words = window_words.cuda()
            labels = labels.cuda()

        #Forward pass
        outputs = model(window_words)
        loss = criterion(outputs, labels)
        loss_epoch.append(loss.item())

        #Obtener metricas del entrenamiento
        y_pred = get_preds(outputs)
        tgt = labels.cpu().numpy()
        training_metric.append(accuracy_score(tgt,y_pred))

        #Backward and optmize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    #Get metric in training dataset
    mean_epoch_metric = np.mean(training_metric)
    train_metric_history.append(mean_epoch_metric)

    #Get metric in validation dataset
    model.eval()
    tuning_metric = model_eval(val_loader, model, gpu=args.use_gpu)
    metric_history.append(mean_epoch_metric)

    #Update scheduler
    scheduler.step(tuning_metric)

    #Check for metric improvement
    is_improvement = tuning_metric > best_metric
    if is_improvement:
        best_metric = tuning_metric
        n_no_improve = 0
    else:
        n_no_improve +=1


    #Save best model if metric improved
    save_checkpoint(
        {
            "epoch": epoch +1,
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "best_metric": best_metric,
        },
        is_improvement,
        args.savedir,
    )

    #Early stopping
    if n_no_improve >= args.patience:
        print("No improvement. Breaking out of loop.")
        break

    print('Train acc:{}'.format(mean_epoch_metric))
    print('Epoch [{}/{}], Loss: {:.4f} - Val accuracy: {:.4f} - Epoch time: {:.2f}'
          .format(epoch+1, args.num_epochs, np.mean(loss_epoch), tuning_metric, (time.time()- epoch_start_time)))

print("--- %s seconds ---" % (time.time() - start_time))

Train acc:0.08024691358024691
Epoch [1/100], Loss: 5.9066 - Val accuracy: 0.0758 - Epoch time: 11.24
Train acc:0.1398533950617284
Epoch [2/100], Loss: 5.0804 - Val accuracy: 0.0825 - Epoch time: 10.49
Train acc:0.1724537037037037
Epoch [3/100], Loss: 4.6572 - Val accuracy: 0.0947 - Epoch time: 9.20
Train acc:0.22260802469135804
Epoch [4/100], Loss: 4.2425 - Val accuracy: 0.1150 - Epoch time: 9.57
Train acc:0.26263503086419754
Epoch [5/100], Loss: 3.9053 - Val accuracy: 0.1367 - Epoch time: 12.04
Train acc:0.31375385802469136
Epoch [6/100], Loss: 3.5025 - Val accuracy: 0.1367 - Epoch time: 11.24
Train acc:0.3505979938271605
Epoch [7/100], Loss: 3.1467 - Val accuracy: 0.0568 - Epoch time: 9.00
Train acc:0.40248842592592593
Epoch [8/100], Loss: 2.9397 - Val accuracy: 0.1069 - Epoch time: 8.80
Train acc:0.4592013888888889
Epoch [9/100], Loss: 2.4614 - Val accuracy: 0.1434 - Epoch time: 8.42
Train acc:0.4994212962962963
Epoch [10/100], Loss: 2.1418 - Val accuracy: 0.1015 - Epoch time: 9.11


### Generador de Texto

In [80]:
def print_closest_words(embeddings, ngram_data, word, n):
    word_id = torch.LongTensor([ngram_data.w2id[word]]) #Obtener id de la palabra
    word_embed = embeddings(word_id) #Obtener embedding de la palabra
    dists = torch.norm(embeddings.weight - word_embed, dim=1).detach() #Calcula las distancias a todas las palabras
    lst = sorted(enumerate(dists.numpy()), key=lambda x: x[1]) #Ordenar por distancia menor a mayor
    for idx, difference in lst[1:n+1]:
        print(ngram_data.id2w[idx], difference)

In [90]:
#Modelo con embeddings aprendidos desde scratch
best_model = NeuralLM(args)
best_model.load_state_dict(torch.load('model/model_best.pt')['state_dict'])
best_model.train(False)

print("-"*30)
print("Learned embeddings")
print("-"*30)
print_closest_words(best_model.emb, ngram_data, "prian", 10)

------------------------------
Learned embeddings
------------------------------
impunidad 11.35109
adelante 11.881222
surgió 12.010161
general 12.044753
humano 12.278279
pregunta 12.281989
público 12.328854
ofrece 12.365208
constitución 12.402827
nuestras 12.517025
